In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} -c estnltk estnltk

# Korpuse tekstide laadimine andmebaasist

Valitud korpuse tekstid salvestatakse TSV-faili 

**Tulbad**:

1. collectionId
2. span.start
3. span.id
4. span.text
5. span.lemma
6. span.upostag
7. span.deprel
8. span.head
9. span.feats



Näide:

NB! Reaalses failis ei ole päist.

|collectionId|span.start|	span.id|	span.text|span.lemma|	span.upostag	|span.deprel|span.head	|span.feats	|
|---|---|---|---|---|---|---|---|---|
|0  |     898  |   1  |     Ma  |    mina  |  P  |     nsubj:cop  |     5  |     sg\|\|nom|
|0  |     901  |   2  |     pole  |  olema |  V  |     cop  |   5  |     ps\|\|sg\|\|aux\|\|neg\|\|ps2\|\|pres\|\|imper|
|0  |     906  |   3  |     enda  |  ise  |   P  |     obl  |   5  |     sg\|\|gen|
|0  |     911  |   4  |     üle  |   üle  |   K  |     case  |  3  |     post|
|0  |     915  |   5  |     uhke  |  uhke  |  A  |     root  |  0  |     sg\|\|nom\|\|pos|
|0  |     920  |   6  |     ainuüksi  |  ainuüksi  |  D  |     advmod  |5  |     |
|0  |     929  |   7  |     oma  |   oma  |   P  |     nmod  |  8  |     sg\|\|gen|
|0  |     933  |   8  |     saavutuste  |    saavutus  |     S  |     obl  |   5  |     pl\|\|com\|\|gen|
|0  |     944  |   9  |     pärast | pärast | K  |     case  |  8  |     post|
|0  |     951  |   10  |    .  |     .  |     Z  |     punct |  5  |     |


## Vajalikud teegid

In [2]:
from __future__ import print_function
#from estnltk import Text, logger
from estnltk.storage.postgres import PostgresStorage
import sqlite3
import sys

## Abifunktsioonid

In [3]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)


Salvestab kollektsioonide "järelhoidja" baasi.

In [4]:
def saveToDisk(rows, lastcollection, fOut):
   
    global TABLENAME, cursor, connection

    fOut.write('\n'.join( '\t'.join(s) for s in rows) + '\n')

    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, lastcollection, lastcollection, ) )
    
    connection.commit()
    eprint(f'TSV-sse salvestatud kollokatsioonid: 0 - {lastcollection}' )

In [5]:
def prepareDb():
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS collections_processed
                      (tablename text, lastcollection integer);
                      """)
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS collections_processed_uniq
    ON collections_processed(tablename);
    """)


In [6]:
def getPlaceholder(tablename):
    cursor.execute(f"""SELECT lastcollection FROM collections_processed WHERE tablename = ?""", (tablename,) );
    lastcollection = cursor.fetchone()

    fileMode = 'w'
    if not lastcollection:
        lastcollection = -1
    else:
        #andmete kogumine jäi pooleli ning jätkatakse samasse faili kirjutamist
        lastcollection = lastcollection[0]
        fileMode = 'a'
    return (lastcollection, fileMode)


## Muutujad

In [7]:
collectionName = 'koondkorpus_base_subset_of_5000_v2' #'koondkorpus_base_v2' # 


Fail, kuhu väljund salvestatakse.

In [8]:
outputfile = f'{collectionName}.tsv'

**Kollektsioonide arv**, mille kaupa salvestatakse vahepealne tulemus faili.

In [9]:
BATCHSIZE = 1000

In [10]:
TABLENAME = f'texts_{collectionName}'

## Andmebaasi loomine ja ette valmistamine

In [11]:
connection = sqlite3.connect("collect_texts.db") #
cursor = connection.cursor()

prepareDb()

## Tekstide alla laadimine

In [12]:
storage = PostgresStorage(pgpass_file='~/.pgpass',
                          schema='estonian_text_corpora',
                          role='estonian_text_corpora_read',
                          temporary=False)

INFO:storage.py:42: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:58: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


In [13]:
collection = storage[collectionName]
TOTALROWS = len(collection)

Kontrollitakse, mitmenda kollektsiooni juurde skript eelmine kord jäi, juhul kui skripti töö katkes ootamatult. Nullist alustamiseks tuleb kustutada skripti loodud *collect_texts.db* fail.

In [14]:
(lastcollection, fileMode) = getPlaceholder(TABLENAME)

#mitu kollektsiooni jäi alla tõmbamata
unprocessed = TOTALROWS - lastcollection - 1

print (f'{unprocessed} out of {TOTALROWS} collections to download')
fOut = open(outputfile, fileMode)

###
iterations = 0
unsaved = 0
rows = []

for (colId, text) in collection.select (progressbar='notebook', layers=['v166_sentences', 'v168_stanza_ensemble_syntax'], return_index=True ).tail(unprocessed):
    sentences_start = [span.start for span in text.v166_sentences]
    sentences_end = [span.end for span in text.v166_sentences]
   
    
    #for morphspan in text.v166_morph_analysis:
    #   
    #    morphforms[morphspan.start] = list(morphspan.form)
    #    morphPOS[morphspan.start] = list(morphspan.partofspeech)
        
    #print (morphforms)
    
    
        
    #continue
    iterations +=1
    unsaved = 1
    for span in text.v168_stanza_ensemble_syntax:
        if span.start in sentences_start:
            current_sentence = []
        # span.text.replace('\n','') - mõned üksikud lemmad koondkorpuses sisaldavad \n sümbolit
        #print (span)
        
        rows.append([str(x) for x in [colId, span.start, span.id, span.text.replace('\n',''), span.lemma, span.upostag, span.deprel, span.head , '||'.join(span.feats.keys()) , ]] )

        current_sentence.append(span)
        if span.end in sentences_end:
            exit()
            unsaved = 1
            continue
    if not iterations%BATCHSIZE:
        saveToDisk(rows, colId, fOut)
        rows = []
        unsaved = 0

if unsaved: saveToDisk(rows, colId, fOut)
fOut.close()


0 out of 5000 collections to download


In [15]:
import subprocess
batcmd=f"wc {outputfile}"
res = subprocess.check_output(batcmd, shell=True)

In [16]:
batcmd


'wc koondkorpus_base_subset_of_5000_v2.tsv'

In [17]:
res

b' 2105386 18425729 98853579 koondkorpus_base_subset_of_5000_v2.tsv\n'